# REGULARISATION
- Regularization consists in adding a penalty on the different parameters of the model to reduce the freedom of the model. Hence, the model will be less likely to fit the noise of the training data and will improve the generalization abilities of the model. For linear models there are in general 3 types of regularisation:
  -  The L1 regularization (also called Lasso)
  -  The L2 regularization (also called Ridge)
  -  The L1/L2 regularization (also called Elastic net)
  
- L1 / Lasso will shrink some parameters to zero, therefore allowing for feature elimination
- For l2 / Ridge, as the penalisation increases, the coefficients approach but do not equal zero, hence no variable is ever excluded

- EMBEDDED METHODS: LASSO:
   - By fitting a linear or logistic regression with a Lasso regularisation, we can then evaluate the coefficients of the different variables, and remove those variables which coefficients are zero.

## Lasso regularisation

Regularisation consists in adding a penalty to the different parameters of the machine learning model to reduce the freedom of the model and avoid overfitting. In linear model regularization, the penalty is applied to the coefficients that multiply each of the predictors. The Lasso regularization or l1 has the property that is able to shrink some of the coefficients to zero. Therefore, those features can be removed from the model.

I will demonstrate how to select features using the Lasso regularisation on a regression and classification problem.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Classification

In [3]:
data = pd.read_csv('../dataset_2.csv')
print(data.shape)
data.head(1)

(50000, 109)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.53271,3.280834,17.982476,4.404259,2.34991,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417


In [4]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['target'], axis=1), data['target'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((35000, 108), (15000, 108))

In [5]:
# linear models benefit from feature scaling
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [7]:
### Select features with Lasso
# here I will do the model fitting and feature selection altogether in one line of code
# first I specify the Logistic Regression model, and I make sure I select the Lasso (l1) penalty.
# Then I use the selectFromModel class from sklearn, which will select the features which coefficients are non-zero
sel_ = SelectFromModel( LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))

sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=LogisticRegression(C=0.5, penalty='l1',
                                             random_state=10,
                                             solver='liblinear'))

In [8]:
# this command let's me visualise the index of the features that were selected
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True, False])

In [9]:
# Now I make a list with the selected features
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format( np.sum(sel_.estimator_.coef_ == 0)))

total features: 108
selected features: 93
features with coefficients shrank to zero: 15


In [10]:
### Examine coefficients that shrank to zero
# the number of features which coefficient was shrank to zero:
np.sum(sel_.estimator_.coef_ == 0)

15

In [11]:
# we can identify the removed features like this:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index(['var_8', 'var_19', 'var_42', 'var_47', 'var_53', 'var_59', 'var_62',
       'var_64', 'var_73', 'var_75', 'var_85', 'var_87', 'var_91', 'var_105',
       'var_109'],
      dtype='object')

In [12]:
# we can then remove the features from the training and testing set
# like this:

X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)

X_train_selected.shape, X_test_selected.shape

((35000, 93), (15000, 93))

Remember that sklearn SelectFromModel returns a NumPy array, so if you need a dataframe, you need to capture the feature names first and then convert the array to a dataframe.

### Ridge regularisation does not shrink coefficients to zero

For the sake of the demo, let's inspect if the Ridge Regularization or L2 shrinks coefficients to zero.

In [13]:
# For comparison, I will fit a logistic regression with a Ridge regularisation, and evaluate the coefficients

l1_logit = LogisticRegression(C=0.5, penalty='l2', max_iter=300, random_state=10)
l1_logit.fit(scaler.transform(X_train), y_train)

# I count the number of coefficients with zero values and it is zero, as expected
np.sum(l1_logit.coef_ == 0)
# play around with the penalty (C) to see if the result changes.

0

## Regression

In [14]:
data = pd.read_csv('../houseprice.csv')
print(data.shape)
data.head(1)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [15]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers, and then you can assess how deterministic they are of the target
# here for simplicity I will use only numerical variables select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [16]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['SalePrice'], axis=1), data['SalePrice'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((1022, 37), (438, 37))

In [17]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [18]:
# the features in the house dataset are in very different scales, so it helps the regression to scale them
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [19]:
# Select Coefficients with Lasso
# here, again I will train a Lasso Linear regression and select the non zero features in one line.
# bear in mind that the linear regression object from sklearn does not allow for regularisation. So If you want to make a regularised
# linear regression you need to import specifically "Lasso"
# alpha is the penalisation, so I set it high to force the algorithm to shrink some coefficients

sel_ = SelectFromModel(Lasso(alpha=100, random_state=10))
sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=Lasso(alpha=100, random_state=10))

In [20]:
sel_.get_support()

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True])

In [21]:
# make a list with the selected features and print the outputs
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format( np.sum(sel_.estimator_.coef_ == 0)))

total features: 37
selected features: 33
features with coefficients shrank to zero: 4


As we can see, both for linear and logistic regression we used the Lasso regularisation to remove non-important features from the dataset. 

Keep in mind that increasing the penalisation will increase the number of features removed. Therefore, you will need to keep an eye and monitor the final model performance to ensure that you don't set a penalty too high so it removes a lot of features, or too low, and thus useless features are retained.

*************************************
**************************************
***************************************

## Basic Filter Methods plus LASSO pipeline
### Putting it all together

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score

In [23]:
data = pd.read_csv('../dataset_1.csv')
data.shape

(50000, 301)

In [24]:
X_train, X_test, y_train, y_test = train_test_split( data.drop(labels=['target'], axis=1), data['target'], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((35000, 300), (15000, 300))

In [25]:
# I keep a copy of the dataset with all the variables to measure the performance of the machine learning models at the end of the notebook
X_train_original = X_train.copy()
X_test_original = X_test.copy()

In [26]:
### Remove constant features
constant_features = [ feat for feat in X_train.columns if X_train[feat].std() == 0 ]
X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 266), (15000, 266))

In [27]:
# Remove quasi-constant features
sel = VarianceThreshold( threshold=0.01)  # 0.1 indicates 99% of observations approximately

sel.fit(X_train)  # fit finds the features with low variance

sum(sel.get_support()) # how many not quasi-constant?

215

In [28]:
features_to_keep = X_train.columns[sel.get_support()]

In [29]:
# remove features
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)
X_train.shape, X_test.shape

((35000, 215), (15000, 215))

In [30]:
# I transform the NumPy arrays to dataframes
X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep
X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [31]:
### Remove duplicated features
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)
    col_1 = X_train.columns[i]
    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
len(duplicated_feat)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


10

In [32]:
# remove duplicated features
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 205), (15000, 205))

In [33]:
# I keep a copy of the dataset without constant, quasi-constant and duplicated variables
# to measure the performance of machine learning models at the end of the notebook
X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

In [34]:
# Remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            # we are interested in absolute coeff value
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)))

correlated features:  93


In [35]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 112), (15000, 112))

In [36]:
# keep a copy of the dataset without correlated features

X_train_corr = X_train.copy()
X_test_corr = X_test.copy()

# Remove features using Lasso

In [37]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [ ]:
# fit a Lasso and selet features, make sure to select l1

sel_ = SelectFromModel(
    LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))

sel_.fit(scaler.transform(X_train), y_train)

# remove features with zero coefficient from dataset
# and parse again as dataframe
X_train_lasso = pd.DataFrame(sel_.transform(X_train))
X_test_lasso = pd.DataFrame(sel_.transform(X_test))

# add the columns name
X_train_lasso.columns = X_train.columns[(sel_.get_support())]
X_test_lasso.columns = X_train.columns[(sel_.get_support())]

In [39]:
X_train_lasso.shape, X_test_lasso.shape

((35000, 90), (15000, 90))

### Compare the performance of Logistic Regression with the different feature subsets

In [40]:
# create a function to train logistic regression and compare performance in train and test set
def run_logistic(X_train, X_test, y_train, y_test):
    # with a scaler
    scaler = StandardScaler().fit(X_train)

    logit = LogisticRegression(random_state=44, max_iter=500)
    logit.fit(scaler.transform(X_train), y_train)

    print('Train set')
    pred = logit.predict_proba(scaler.transform(X_train))
    print('Logistic Regression roc-auc: {}'.format(  roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = logit.predict_proba(scaler.transform(X_test))
    print('Logistic Regression roc-auc: {}'.format( roc_auc_score(y_test, pred[:, 1])))

In [41]:
# original
run_logistic(X_train_original, X_test_original, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8028301567886174
Test set
Logistic Regression roc-auc: 0.7951109867720157


In [42]:
# filter methods - basic
run_logistic(X_train_basic_filter, X_test_basic_filter, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.802270249932219
Test set
Logistic Regression roc-auc: 0.7947366130595612


In [43]:
# filter methods - correlation
run_logistic(X_train_corr, X_test_corr, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7942670292952432
Test set
Logistic Regression roc-auc: 0.7881803821166471


In [44]:
# embedded methods - Lasso
run_logistic(X_train_lasso, X_test_lasso, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7941658764642089
Test set
Logistic Regression roc-auc: 0.788232378465599


 we reduced the feature space quite a bit, without losing model performance dramatically.